In [183]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import numpy as np
import pandas as pd
import wandb
import plotly.express as px

In [184]:
wandb.login()

True

In [185]:
wandb.init(
    # set the wandb project where this run will be logged
    project="assignment-1",
    name="pytorch",
    reinit=True,
    
    # track hyperparameters and run metadata
    config={
        "learning_rate": 1e-3,
        "dataset": "Titanic",
        "epochs": 500,
        "loss": "binary cross-entropy",
        "optimizer": "Adam"
    }
)
config = wandb.config

In [186]:
# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

Using cpu device


# Pytorch implementation

## Loading the dataset

In [187]:
class TitanicDataset(Dataset):
    def __init__(self, data_dir):
        self.labels = pd.read_csv(os.path.join(data_dir, 'labels.csv')).to_numpy(dtype='float32')
        self.data = pd.read_csv(os.path.join(data_dir, 'data.csv')).to_numpy(dtype='float32')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # print(self.data[idx])
        data = torch.from_numpy(self.data[idx])
        label = torch.from_numpy(self.labels[idx])
        # data = self.data[idx]
        # label = self.labels[idx]

        return data, label

In [188]:
train_data = TitanicDataset(data_dir='data/train/')
val_data = TitanicDataset(data_dir='data/val/')
test_data = TitanicDataset(data_dir='data/test/')

In [189]:
features, label = train_data.__getitem__(idx=0)
print(features, label)

tensor([  1.0000,  49.0000,   1.0000,   1.0000, 110.8833]) tensor([0.])


In [190]:
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

# train_x, train_y = next(iter(train_dataloader))
# print(train_x.shape, train_y.shape)

In [191]:
for batch, (x, y) in enumerate(train_dataloader):
    print(batch, x.shape, y.shape)
    break

0 torch.Size([16, 5]) torch.Size([16, 1])


## Defining the model

In [192]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.linear = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear(x)
        return logits

model = NeuralNetwork().to(device)
wandb.config['model'] = model.__dict__['_modules']['linear']
print(model)

NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
    (7): Sigmoid()
  )
)


In [193]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

In [194]:
wandb.watch(model, loss_fn, log='all', log_freq=10)

[]

In [195]:
features, label = train_data.__getitem__(idx=0)
features = features.to(device)
label = label.to(device)

pred = model(features)
print('pred: ', pred)
print('loss: ', loss_fn(pred, label))

pred:  tensor([0.5573], grad_fn=<SigmoidBackward0>)
loss:  tensor(0.8149, grad_fn=<BinaryCrossEntropyBackward0>)


In [196]:
def calculate_acc(y_pred, y) -> float:
    y_pred = y_pred>0.5

    # print(((y == y_pred).sum().item()))
    return ((y == y_pred).sum().item())/len(y)

In [197]:
loss_graph = []
acc_graph = []

def train(dataloader, model, loss_fn, optimizer, epoch):
    curr = 0
    avg_acc = 0
    avg_loss = 0
    model.train()

    for x, y in dataloader:
        curr += len(y)

        x = x.to(device)
        y = y.to(device)

        # Get prediction and compute loss
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        avg_loss += loss.item()

        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # correct = (pred == y).type(torch.float).sum().item()/len(y)
        avg_acc += calculate_acc(y_pred, y)

        # print(f'loss: {loss.item():>5f} acc: {100*correct:>5f} | {curr}/{len(dataloader.dataset)}')

    avg_acc /= len(dataloader)
    avg_loss /= len(dataloader)
    loss_graph.append(avg_loss)
    acc_graph.append(avg_acc)
    
    # if (epoch % 10):
    print(f'avg loss: {avg_loss:>5f} avg acc: {avg_acc:>5f}')
    
    wandb.log({'loss': avg_loss})
    wandb.log({'acc': avg_acc})

In [198]:
val_loss_graph = []
val_acc_graph = []

def val(dataloader, model, loss_fn, epoch):    
    model.eval()
    avg_loss = 0
    avg_acc = 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            avg_loss += loss_fn(y_pred, y).item()
            avg_acc += calculate_acc(y_pred, y)
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)

    val_loss_graph.append(avg_loss)
    val_acc_graph.append(avg_acc)
    
    # if (epoch % 10):
    print(f'avg val loss: {avg_loss:>5f} avg val acc: {avg_acc:>5f}')
    
    wandb.log({'val_loss': avg_loss})
    wandb.log({'val_acc': avg_acc})

In [199]:
epochs = config['epochs']
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    train(train_dataloader, model, loss_fn, optimizer, epoch)
    val(val_dataloader, model, loss_fn, epoch)
    print('-------------------------------')
print('Done!')

Epoch 1
avg loss: 0.632256 avg acc: 0.666667
avg val loss: 0.645051 avg val acc: 0.620833
-------------------------------
Epoch 2
avg loss: 0.613280 avg acc: 0.681624
avg val loss: 0.637819 avg val acc: 0.686111
-------------------------------
Epoch 3
avg loss: 0.619711 avg acc: 0.698184
avg val loss: 0.618763 avg val acc: 0.629167
-------------------------------
Epoch 4
avg loss: 0.603616 avg acc: 0.705128
avg val loss: 0.622684 avg val acc: 0.650000
-------------------------------
Epoch 5
avg loss: 0.603434 avg acc: 0.694979
avg val loss: 0.616946 avg val acc: 0.651389
-------------------------------
Epoch 6
avg loss: 0.604171 avg acc: 0.697115
avg val loss: 0.629434 avg val acc: 0.672222
-------------------------------
Epoch 7
avg loss: 0.606384 avg acc: 0.714744
avg val loss: 0.656078 avg val acc: 0.608333
-------------------------------
Epoch 8
avg loss: 0.605234 avg acc: 0.676816
avg val loss: 0.636416 avg val acc: 0.690278
-------------------------------
Epoch 9
avg loss: 0.5987

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/Users/kuko/miniconda3/envs/cv/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/kuko/miniconda3/envs/cv/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/kuko/miniconda3/envs/cv/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/Users/kuko/miniconda3/envs/cv/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/Users/kuko/miniconda3/envs/cv/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


avg loss: 0.560674 avg acc: 0.729167
avg val loss: 0.568614 avg val acc: 0.690278
-------------------------------
Epoch 29
avg loss: 0.564518 avg acc: 0.724893
avg val loss: 0.570099 avg val acc: 0.713889
-------------------------------
Epoch 30
avg loss: 0.554932 avg acc: 0.715812
avg val loss: 0.619501 avg val acc: 0.677778
-------------------------------
Epoch 31
avg loss: 0.559874 avg acc: 0.727564
avg val loss: 0.588160 avg val acc: 0.705556
-------------------------------
Epoch 32
avg loss: 0.557248 avg acc: 0.729167
avg val loss: 0.584404 avg val acc: 0.705556
-------------------------------
Epoch 33
avg loss: 0.549117 avg acc: 0.730235
avg val loss: 0.575442 avg val acc: 0.712500
-------------------------------
Epoch 34
avg loss: 0.550453 avg acc: 0.726496
avg val loss: 0.572022 avg val acc: 0.698611
-------------------------------
Epoch 35
avg loss: 0.552540 avg acc: 0.729167
avg val loss: 0.553815 avg val acc: 0.713889
-------------------------------
Epoch 36
avg loss: 0.5553

In [200]:
wandb.finish()

acc,▁▂▃▃▂▃▃▃▃▃▃▃▄▄▄▄▅▄▄▅▅▆▆▆▆▇▆▇▇▆▇▇█▇▇▇▇█▇█
loss,█▇▇▆▆▆▆▆▅▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁
val_acc,▁▅▃▃▅▄▅▄▃▅▆▇▅▆▄▅▅▃▇▃▅▄▅▆▅▆▆▅▅▃▅▆▇▅▄▅▄▅▄█
val_loss,▅▃▂▄▂▂▃▂▃▁▁▁▁▂▅▃▃▅▂▅▂▆▅▁▃▆▄▄▆▅▅▅▆▇▇▇█▆█▇
acc,0.80556
loss,0.40504
val_acc,0.72083
val_loss,0.70479


In [201]:
fig = px.line({'loss': loss_graph, 'val loss': val_loss_graph})
fig.show()

In [202]:
fig = px.line({'acc': acc_graph, 'val acc': val_acc_graph})
fig.show()

In [203]:
for batch, (x, y) in enumerate(test_dataloader):
    print(batch)
    x = x.to(device)
    y = y.to(device)

    pred = model(x)
    # print('pred: ', pred)
    # print('===========')
    # print('pred>0.5: ', pred>0.5)
    # print('===========')
    # print('y: ', y)
    # print('===========')
    print(calculate_acc(pred, y))
    print('loss: ', loss_fn(pred, y))
    break

0
0.75
loss:  tensor(6.7898, grad_fn=<BinaryCrossEntropyBackward0>)
